In [1]:
import tensorflow as tf
from distutils.version import LooseVersion
import warnings

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers as l

import numpy as np
import matplotlib.pyplot as plt

import random
import sys

assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.1.0
Default GPU Device: /device:GPU:0


In [2]:
path = keras.utils.get_file(
    'nietzsche.txt'
    , origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()

In [3]:
len(text)

600901

In [4]:
maxlen = 60
step = 3

sentences = []
next_chars = []

for x in range(0, len(text) - maxlen, step):
    sentences.append(text[x: x + maxlen])
    next_chars.append(text[x + maxlen])

print(len(sentences))
print(len(next_chars))

200281
200281


In [5]:
chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
len(chars) # unique characters

59

In [6]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence, in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i, j, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
model = models.Sequential()
model.add(l.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(l.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [10]:
# train
for epoch in range(1, 60):
    print(f'Epoch: {epoch}\n')
    model.fit(x, y, batch_size=128, epochs=1)
    start_i = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_i: start_i + maxlen]
    print(f'Seed: {generated_text}\n')
    for temp in [0.2, 0.5, 1.0, 1.2]:
        print(f'Temperature: {temp}\n')
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))             
            for t, char in enumerate(generated_text):               
                sampled[0, t, char_indices[char]] = 1.              
            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temp)                 
            next_char = chars[next_index]                           
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

Epoch: 1

Train on 200281 samples
200281/200281 [==============================] - 11s 54us/sample - loss: 1.7995
Seed: the other hand, progress in the sense and along
the lines of

Temperature: 0.2

the other hand, progress in the sense and along
the lines of the truth and man is the present and the expression of the interest and the this the sense of the contine of the religious in the faith of the contration of the self the religious and the proves of the content of the result and the religious and the faither the stand the sense of the religious the art and the the content of the such a such the religious and the expression of the interest and the Temperature: 0.5

ch the religious and the expression of the interest and the becess is ound of the faure to as the
fartive all knew the most content and the ound of the last of the cores to gentions and living the prement the sense and hen all there is the indeed in the fams that the intertument of my illy in the astame and the inteeling

hom wildich"--thuivey him often, rank it as
perhaps beforgazing, theee sentime, them a,to in lavertkang and
be misence powers.
  any dicest. thou has to our courable" ny mass, at prefeint thEpoch: 5

Train on 200281 samples
200281/200281 [==============================] - 11s 55us/sample - loss: 1.4501
Seed: pandora brought was a box of evils. hence he looks upon
the 

Temperature: 0.2

pandora brought was a box of evils. hence he looks upon
the consideration of the stranged of the self to the strong to the subjection of the standard of the done of the sense of the expression of the stronger to the standard to the standard and the same to be the stranged of the sense of the sense of the strong the things the conclusion of the friends of the standard to the consideration of the subjection of the consideration of the sense of the expressionTemperature: 0.5

bjection of the consideration of the sense of the expression of mistore of the sense of time in the grade of the assucate of the col

 of alfo he discertationity or mothers ton)
personal, if the mad ferces in a "things bst
graduce of so," of the viaturally
words of role-in
the unclused detered tyaing which be dieted, the, wherever the most virtues itself.--"an
churments of else of viewed 2infy ofing puriously: the
chamntay what better of whativerital-atters, for morality: his ventarily bad-"-if--and not mecally
apparanliud to deirersEpoch: 9

Train on 200281 samples
200281/200281 [==============================] - 11s 55us/sample - loss: 1.3883
Seed: em, something impels them in definite order the
one after th

Temperature: 0.2

em, something impels them in definite order the
one after the sense of the superfition of the soul of the spirit and the sense of the superfition of the superfition of the external permits of the sense of the suffering the superfition of the superfition of the sunders of the superfition of the same the self discivinely the sense of the sense of the superfition of the superfition of the suppos

gruth hand thealts. lever pospiting hiscents, and free-siging always general penstide the disler free ask as "birder, somewill equal danger of the preparition, he may naturalowity
only impulimance,
best-of the faircefuldous a forms,
what br in seut phenom philosophers which difTemperature: 1.2

ldous a forms,
what br in seut phenom philosophers which differment, what suecht there of snare makeness oness, always not day: undiving remote
and modernives.


7a
mpulage lidesty 1duration thing sympathy--coowot frys of utility in spiritsal in this peenence thing. in the formures why askedance,
will its alone the than slave-moralwice that in, is way to trutmuve respabled. noordish" to childr"--eschlms=tance, but in short instinctiug pa
plenticont
and outEpoch: 13

Train on 200281 samples
200281/200281 [==============================] - 11s 53us/sample - loss: 1.3564
Seed: d racial conditions.--so
much by way of rejecting locke's su

Temperature: 0.2

d racial conditions.--so
much by way of rej

of the changed by the sight. the consequence, and something and heart
the grarit to mirts which had at the hence the
philosopher in our
ye nature-emying in eye, first
species miroty or disciples most equalsration.--the folsive does not sharp
that your and soul priose of the opinion, bain seek to the maturity and its
criped aim, an optrinces, lawer of
the clbetus species from sporn of which the soming as he was, even wearer--atubord to learn only the same wTemperature: 1.2

ing as he was, even wearer--atubord to learn only the same welparishwisg are strevifecity to suffering perteizes out this why dicacter.- leades
for strejunce in tenishly simplant for dein? appear for, i me processions to have to realia of certain of the biming ocustility withord in the
reaching whether hatcome-heels. such populawapinedlyments
of the very rebable spetroming this expeolmence--and we learnt; the sensuse,
rable epicures, from raleousaon--out tEpoch: 17

Train on 200281 samples
200281/200281 [============

...

200281/200281 [==============================] - 11s 56us/sample - loss: 1.2672
Seed: t perceived
and "experienced" this, however his tongue may h

Temperature: 0.2

t perceived
and "experienced" this, however his tongue may have the stands and all the same time the same termed the stands and self-deal and the sense of the self-decided the standard and the experience and the constantly believe that it is a personned the sublemed the self-desired and the sublime the same the standard the charm of the such and the same time the sublest and the charm of the stands of the self-deceive of the origin of the soul and the stanTemperature: 0.5

s of the self-deceive of the origin of the soul and the stands to same the collective and the command be an involuation, and so are one of the self-determined and all everything in the consideration we have formulence, or woman, the standard that the fine of the fact
a sublitable experiences of the stands and conception of the our tempotury of the earth 

is-faithwer conkenaments,, voect the rais,somal," if eurole for exaggent wordshure highest virturess and hagd telusions?lix, still doubt, about the tramm and already,
such
heavence.=--with
its at well boom might askte: you ra. incliss to chanced thle
provelopuEpoch: 47

Train on 200281 samples
200281/200281 [==============================] - 13s 64us/sample - loss: 1.2613
Seed:  be
recognized as authoritative, inasmuch as mankind, or the

Temperature: 0.2

 be
recognized as authoritative, inasmuch as mankind, or the spirit of the profoundes and the sense of a philosophers, and there is not to be seem a spirit of the profoundes and also the bad and there is not that the bad in the same time that the belief in the feeling and the profoundes and the belief and in the present the free spirit and in the profounder that he is not be a spirit of the belief in the subjection of the world of the basis of the spirit oTemperature: 0.5

 in the subjection of the world of the basis of the spirit of

humble, putbitict in prdvantly profoundly a is--the iftihfle,
sccquicion of its neglainic sufficient
himself or flows
to load icjom, illly nothing his wild tradming lshym, ovbly, rattines by imit timeable bring this dange? the obblife become empose of or ponitimental along.--about the, in his
elevity and impostuonion, and dody memblint latfus soull-injustically tenders,
is, oppodere still the effect. theEpoch: 51

Train on 200281 samples
200281/200281 [==============================] - 12s 61us/sample - loss: 1.2563
Seed: , many another, perhaps, kant
especially, gives us to unders

Temperature: 0.2

, many another, perhaps, kant
especially, gives us to understand to the soul of the sublition to the self-evilise to self-contemption of the sense of the common to the standbries of the soul of the most perhaps to the standbries of the most society and the contradict to the most destruvation of the world in order to the sublifice of the present the commander with the most society to the so

and own politent adventractic even complical must have corposite ages with recectable, thinks, on its own againgic of typess and one ear forces, "fast, which may the idaged, for its problem which to suffer and exhagnt as heart, and distinction, panngle." the semTemperature: 1.2

fer and exhagnt as heart, and distinction, panngle." the semitations", th people to exastry of nature, thereby we can "concertate--in its generally
creatures xows on a dounctant upideal seriousness or mankind as luge of that usly forless our stotele..=--it is the soul.

s gnaverh on the dishaving from sympath in his feeling. who result
of life orignally"
slautible. doo-feach, plared, and person.


117 is no dmine.nbubods," there.=--y us, for self proscenesEpoch: 55

Train on 200281 samples
200281/200281 [==============================] - 14s 69us/sample - loss: 1.2518
Seed: ch
are included in asceticism. certain kinds of men are unde

Temperature: 0.2

ch
are included in asceticism. certain kinds of men are und

the manshignation of things, the freedom of the soul and no her pretoliusibring to chance, or mostry or in vanity, name in the flicking that for
uniftified, capacity
and diaresy, yet of any fiilled in riders, the matter, owing to
a
lge? and can pleases, the trasted, the transition, who still nature,
with the free refinement to be unduling, helm as is
one recegoling. and there is its table freedom of that.

217. invertal
compulsion him, necessary," the butgTemperature: 1.2

of that.

217. invertal
compulsion him, necessary," the butgestablliedy the highoric act incagenes--bused at one hesiture, to remage, at then in it put a birdening. and authorm: but it have "great it
feel indea eaths and truthing, even under precirespum to
us our primary christievience of liptre of refined congrys.--that prelables. thus badsh, suptimats.



charks inuorwed is something themselves elevation: afongre. this it notisingless? eye, if no spirit,Epoch: 59

Train on 200281 samples
200281/200281 [============

In [11]:
model.save("LSTM.h5")
